<a href="https://colab.research.google.com/github/junior8801/Web_Scraping_CVM_Py_Selenium/blob/main/Web_Scraping_CVM_Py_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Web Scraping Composição do Índice Bovespa

Fonte: https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br


##Bibliotecas

In [1]:
!pip install -q kora

     |████████████████████████████████| 57 kB 2.3 MB/s 
     |████████████████████████████████| 56 kB 3.0 MB/s 


In [2]:
!pip install -U plotly

     |████████████████████████████████| 22.9 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [3]:
from kora.selenium import wd
import pandas as pd
import numpy as np
from google.colab.data_table import DataTable
from time import sleep
import plotly.express as px

##Loading
Baixar composição do Ibovespa

In [4]:
tempo = 10
url_base = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br'
wd.get(url_base)
sleep(tempo)
wd.find_element_by_id('segment').send_keys("Setor de Atuação")
sleep(tempo)
wd.find_element_by_link_text('Download').click()
sleep(tempo*3)
arquivos = !ls -1t *.csv

In [11]:
ibov = pd.read_csv(arquivos[0],sep=';',encoding='ISO-8859-1',skipfooter=2,engine='python',thousands='.',decimal=',',header=1,index_col=False)

##Preparação dos Dados


DataSet original

In [12]:
ibov.head(5)

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1482296002,2.523,2.523
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,736143105,0.791,0.791
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327262616,0.575,2.615
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,1115693556,0.653,2.615
4,Bens Indls/Transporte,ECOR3,ECORODOVIAS,ON NM,330237914,0.155,2.615


In [14]:
ibov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Setor           84 non-null     object 
 1   Código          84 non-null     object 
 2   Ação            84 non-null     object 
 3   Tipo            84 non-null     object 
 4   Qtde. Teórica   84 non-null     int64  
 5   Part. (%)       84 non-null     float64
 6   Part. (%)Acum.  84 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.7+ KB


Separar a informação de setor e subsetor

In [15]:
ibov['SubSetor'] = ibov['Setor'].apply(lambda s: s[s.rfind("/")+1:].strip())
ibov['Setor'] = ibov['Setor'].apply(lambda s: s[:s.rfind("/")].strip())

Ajuste setores duplicados

In [16]:
ibov.loc[ibov.Setor == 'Cons N  Básico' ,  'Setor'] = 'Consumo Não-Cíclico'
ibov.loc[ibov.Setor == 'Cons N Cíclico',  'Setor'] = 'Consumo Não-Cíclico'
ibov.loc[ibov.Setor == 'Financ e Outros',  'Setor'] = 'Financeiro e Outros'


DataSet ajustado

In [19]:
ibov.Setor.unique()

array(['Bens Indls', 'Consumo Não-Cíclico', 'Consumo Cíclico', 'Diverso',
       'Financeiro e Outros', 'Mats Básicos', 'Petróleo', 'Saúde',
       'Tec.Informação', 'Telecomunicaçã', 'Utilidade Públ'], dtype=object)

Representação gráfica
1. Gráfico de rosca

In [8]:
fig = px.sunburst(data_frame=ibov, path=['Setor','SubSetor','Código'], values='Part. (%)', height=900)
fig.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:.2f}%')
fig.show()

2. Gráfico de árvore

In [9]:
fig = px.treemap(data_frame=ibov, path=['Setor','SubSetor','Código'], values='Part. (%)', height=900)
fig.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:.2f}%')
fig.show()